### Policy

In [ ]:
reduced = listings[["id", "is_business_travel_ready", "cancellation_policy", "require_guest_profile_picture", "require_guest_phone_verification"]]

In [ ]:
policy_mapping = {}
for i, e in enumerate(reduced.cancellation_policy.unique()):
    policy_mapping[e] = i

In [ ]:
policy_id = np.arange(len(listings.id))

In [ ]:
reduced.insert(1, column="policy_id", value=policy_id)

In [ ]:
for i in reduced.index:
    reduced.at[i, "policy_id"] = int(policy_mapping.get(reduced.at[i, "cancellation_policy"]))

In [ ]:
[print("Hey") for i in reduced.index if reduced.at[i, "is_business_travel_ready"] == "t"]

In [ ]:
reduced.to_sql("Policy", engine, if_exists="append", index=False, dtype={"id" : sqlalchemy.INT, "is_business_travel_ready" : sqlalchemy.CHAR(1), "cancellation_policy" : sqlalchemy.CHAR(32), "require_guest_profile_picture" : sqlalchemy.CHAR(1), "require_guest_phone_verification" : sqlalchemy.CHAR(1)})

## Calendar

In [ ]:
cal = pd.DataFrame({"cal_id" : np.arange(len(calendar.date.unique())), "date" : calendar.date.unique()})

In [ ]:
for i in cal.index:
    cal.at[i, "date"] = datetime.datetime.strptime(str(cal.at[i, "date"]), "%Y-%m-%d").date()

In [ ]:
cal.to_sql("Calendar", engine, if_exists="append", index=False, dtype={"cal_id" : sqlalchemy.INT, "date" : sqlalchemy.DATE})

## Available at

In [ ]:
date_mapping = {}
for i in cal.index:
    date_mapping[str(cal.at[i, "date"])] = cal.at[i, "cal_id"]

In [ ]:
second_mapping = []
for i in calendar.index:
    second_mapping.append(date_mapping.get(calendar.at[i, "date"]))

In [ ]:
avail = pd.DataFrame({"id" : calendar.listing_id, "cal_id" : second_mapping, "price" : calendar.price, "available" : calendar.available})

In [ ]:
for i in avail.index:
    if not pd.isna(avail.at[i, "price"]):
        avail.at[i, "price"] = float(str(avail.at[i, "price"]).replace(",","").replace("$", ""))

In [ ]:
avail = avail.fillna(-1)

In [ ]:
avail.to_sql("Available_at", engine, if_exists="append", index=False, dtype={"id" : sqlalchemy.INT, "cal_id" : sqlalchemy.INT, "price" : sqlalchemy.INT, "available" : sqlalchemy.CHAR(1)})

## Verification

In [ ]:
verification = pd.DataFrame({"host_verifications_id" : np.arange(len(new_list)), "host_verifications" : new_list})